# Resources

https://nlp.stanford.edu/courses/cs224n/2004/may-steinberg-project.pdf  
http://cogcomp.org/Data/QA/QC/  
https://www.nltk.org/api/nltk.classify.html?highlight=maxent  

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.classify.maxent import MaxentClassifier, MaxentFeatureEncodingI, accuracy
from nltk.stem import WordNetLemmatizer

# This uses corenlp server, but changing it to use jar files shouldn't be too much work
from nltk.tag.stanford import CoreNLPNERTagger
from nltk.tag.stanford import CoreNLPPOSTagger

ner_tagger = CoreNLPNERTagger(url='http://localhost:9000')
pos_tagger = CoreNLPPOSTagger(url='http://localhost:9000')

wordnet_lemmatizer = WordNetLemmatizer()

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.classify.maxent import MaxentClassifier, MaxentFeatureEncodingI, accuracy
import spacy

nlp = spacy.load('en')

# Load data

In [2]:
corpus = list()

with open('../data/input/train_5500.txt', 'r') as f:
    for line in f:
        corpus.append(line[:-1].split())
        
print('Training corpus length:', len(corpus))

labels = [line.pop(0) for line in corpus]

Training corpus length: 5452


In [3]:
corpus_test = list()

with open('../data/input/trec_10.txt', 'r') as f:
    for line in f:
        corpus_test.append(line[:-1].split())
        
print('Test corpus length:', len(corpus_test))

labels_test = [line.pop(0) for line in corpus_test]

Test corpus length: 500


# Feature creation

This is where a lot of the hard work happens. Common features include:
* lemma / stem
* POS
* NER
* parse trees
* WordNet features
* bigrams

In [ ]:
# Lazy feature creation - nltk
def create_features(question):
    ner_tags = [i[1] for i in ner_tagger.tag(question)]
    pos_tags = [i[1] for i in pos_tagger.tag(question)]
    
    features = {}
    features['person'] = 'PERSON' in ner_tags
    features['proper_noun'] = 'NNP' in pos_tags
    features['lemma_1'] = wordnet_lemmatizer.lemmatize(question[0])
    return features

In [37]:
# Lazy feature creation - spacy
def create_features(doc):
    pos_tags = [token.pos_ for token in doc]
    ner_tags = [ent.label_ for ent in doc.ents]
    lower = [token.lower_ for token in doc]
    
    features = {}
    features
    features['person'] = 'PERSON' in ner_tags
    features['ne_1'] = ner_tags[0] if ner_tags else ''
    features['proper_noun'] = ('NNP' or 'NNPS') in pos_tags
    features['length'] = len(doc)
    features['lemma_1'] = doc[0].lemma_
    
    if 'VERB' in pos_tags:
        features['verb_1'] = doc[pos_tags.index('VERB')].lemma_
    else:
        features['verb_1'] = ''
        
    features['who'] = 'who' in lower
    features['when'] = 'who' in lower
    
    return features

# Training

#### NLTK

In [ ]:
questions = [(question, label) for question, label in zip(corpus, labels)]

feats = [(create_features(q), l) for (q, l) in questions]

In [64]:
questions_test = [(question, label) for question, label in zip(corpus_test, labels_test)]

feats_test = [(create_features(q), l) for (q, l) in questions_test]

#### Spacy

In [38]:
questions = [(question, label) for question, label in zip(corpus, labels)]
    
feats = [(create_features(nlp.make_doc(' '.join(q))), l) for (q, l) in questions]

In [39]:
questions_test = [(question, label) for question, label in zip(corpus_test, labels_test)]

feats_test = [(create_features(nlp.make_doc(' '.join(q))), l) for (q, l) in questions_test]

#### Train

In [40]:
maxent_classifier = MaxentClassifier.train(feats, max_iter=10, trace=3)

  ==> Training (10 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -3.91202        0.002
             2          -2.39899        0.218
             3          -2.18958        0.357
             4          -2.04344        0.391
             5          -1.93387        0.399
             6          -1.84830        0.404
             7          -1.77924        0.405
             8          -1.72211        0.406
             9          -1.67396        0.406
         Final          -1.63281        0.407


# Results

In [41]:
accuracy(maxent_classifier, feats_test)

0.47

In [35]:
predictions = maxent_classifier.classify_many([fs for (fs, l) in feats_test])

pd.options.display.max_columns = 100
crosstab = pd.crosstab(np.array(predictions), np.array([l for (fs, l) in feats_test]))
crosstab.style.apply(lambda x: ['background: lightblue' if x.name == i else '' 
                                for i,_ in x.iteritems()])

col_0,ABBR:abb,ABBR:exp,DESC:def,DESC:desc,DESC:manner,DESC:reason,ENTY:animal,ENTY:body,ENTY:color,ENTY:currency,ENTY:dismed,ENTY:event,ENTY:food,ENTY:instru,ENTY:lang,ENTY:other,ENTY:plant,ENTY:product,ENTY:sport,ENTY:substance,ENTY:techmeth,ENTY:termeq,ENTY:veh,HUM:desc,HUM:gr,HUM:ind,HUM:title,LOC:city,LOC:country,LOC:mount,LOC:other,LOC:state,NUM:count,NUM:date,NUM:dist,NUM:money,NUM:other,NUM:perc,NUM:period,NUM:speed,NUM:temp,NUM:weight
row_0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DESC:def,1,8,123,4,0,2,10,0,6,4,2,0,2,1,1,4,4,0,0,12,0,3,1,0,1,4,0,9,0,0,11,2,0,8,1,0,9,0,0,2,1,1
DESC:manner,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,3,0,0,0,0,2,1,1
DESC:reason,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
HUM:ind,0,0,0,3,0,0,6,2,4,2,0,1,2,0,1,8,0,4,1,3,1,4,2,3,4,51,1,8,3,2,13,4,0,11,6,1,2,2,2,1,2,1
LOC:city,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
LOC:other,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,24,0,0,0,0,0,0,0,0,0,0,0
NUM:count,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5,2,6,2,1,1,5,1,0,1
NUM:date,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,1,0,26,0,0,0,0,0,0,0,0


In [43]:
import pickle

with open('../models/maxent_classifier.pkl', 'wb') as f:
    pickle.dump(maxent_classifier, f)

In [7]:
import pickle

with open('../models/maxent_classifier.pkl', 'rb') as f:
    maxent_classifier = pickle.load(f)